## BRIA Background Removal v1.4 Model
#### https://huggingface.co/briaai/RMBG-1.4

In [1]:
import json

In [2]:
from BRIA_utils import save_BRIA_image, run_batched_BRIA_processing

In [3]:
pip install -qr https://huggingface.co/briaai/RMBG-1.4/resolve/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


#### TEST

In [ ]:
with open("../objects_db_test.json", "r") as f:
    test_data = json.load(f)

In [ ]:
# print(len(test_data['objects']))
test_objs = test_data['objects']

In [ ]:
test_output_path = "../public/data/initial_test/BRIA/"

In [ ]:
save_BRIA_image(test_objs[0], test_output_path)

In [ ]:
run_batched_BRIA_processing(test_objs, test_output_path)

### Save images to folder for FE

In [5]:
with open("../objects_db.json", "r") as f:
    data = json.load(f)

objs = data['objects']

output_path = "../public/data/images/"

In [6]:
save_BRIA_image(objs[0], output_path)

Skipped: ../public/data/images/14318mask.png already exists.


In [7]:
save_BRIA_image(objs[0], output_path)

Skipped: ../public/data/images/14318mask.png already exists.


In [8]:
run_batched_BRIA_processing(objs, output_path)

Processing batches:   0%|          | 0/227 [00:00<?, ?it/s]

Skipped: ../public/data/images/14318mask.png already exists.
Skipped: ../public/data/images/9480mask.png already exists.
Skipped: ../public/data/images/16589mask.png already exists.
Skipped: ../public/data/images/17732mask.png already exists.
Skipped: ../public/data/images/551250mask.png already exists.
Skipped: ../public/data/images/21355mask.png already exists.
Skipped: ../public/data/images/820892mask.png already exists.
Skipped: ../public/data/images/42229mask.png already exists.
Skipped: ../public/data/images/9317mask.png already exists.
Skipped: ../public/data/images/19067mask.png already exists.
Skipped: ../public/data/images/208181mask.png already exists.
Skipped: ../public/data/images/20804mask.png already exists.
Skipped: ../public/data/images/9361mask.png already exists.
Skipped: ../public/data/images/6779mask.png already exists.
Skipped: ../public/data/images/6778mask.png already exists.
Skipped: ../public/data/images/1029mask.png already exists.
Skipped: ../public/data/ima

Device set to use mps:0
Processing batches:  33%|███▎      | 75/227 [00:03<00:07, 20.52it/s]

Error processing object 238928: cannot identify image file <_io.BytesIO object at 0x358781260>
Skipped: ../public/data/images/250258mask.png already exists.
Skipped: ../public/data/images/243971mask.png already exists.
Skipped: ../public/data/images/52937mask.png already exists.
Skipped: ../public/data/images/52938mask.png already exists.
Skipped: ../public/data/images/550071mask.png already exists.
Skipped: ../public/data/images/549375mask.png already exists.
Skipped: ../public/data/images/248849mask.png already exists.
Skipped: ../public/data/images/204619mask.png already exists.
Skipped: ../public/data/images/204620mask.png already exists.
Skipped: ../public/data/images/770924mask.png already exists.
Skipped: ../public/data/images/44247mask.png already exists.
Skipped: ../public/data/images/62470mask.png already exists.
Skipped: ../public/data/images/468422mask.png already exists.
Skipped: ../public/data/images/675791mask.png already exists.
Skipped: ../public/data/images/449686mask

Device set to use mps:0
Processing batches:  55%|█████▌    | 125/227 [00:05<00:03, 25.74it/s]

...

Skipped: ../public/data/images/827419mask.png already exists.
Skipped: ../public/data/images/852636mask.png already exists.
Skipped: ../public/data/images/665746mask.png already exists.
Skipped: ../public/data/images/677216mask.png already exists.
Skipped: ../public/data/images/842983mask.png already exists.
Skipped: ../public/data/images/42147mask.png already exists.
Skipped: ../public/data/images/206454mask.png already exists.
Skipped: ../public/data/images/42205mask.png already exists.
Skipped: ../public/data/images/42222mask.png already exists.
Skipped: ../public/data/images/453244mask.png already exists.
Skipped: ../public/data/images/207383mask.png already exists.
Skipped: ../public/data/images/203141mask.png already exists.
Skipped: ../public/data/images/834813mask.png already exists.
Skipped: ../public/data/images/834812mask.png already exists.
Skipped: ../public/data/images/203140mask.png already exists.
Skipped: ../public/data/images/42223mask.png already exists.
Skipped: ../

Device set to use mps:0
Processing batches:  98%|█████████▊| 223/227 [30:32<00:04,  1.22s/it]

Skipped: ../public/data/images/231854mask.png already exists.
Skipped: ../public/data/images/231855mask.png already exists.
Skipped: ../public/data/images/770967mask.png already exists.
Skipped: ../public/data/images/50714mask.png already exists.
Skipped: ../public/data/images/49362mask.png already exists.
Skipped: ../public/data/images/50349mask.png already exists.
Skipped: ../public/data/images/199342mask.png already exists.
Skipped: ../public/data/images/200854mask.png already exists.
Skipped: ../public/data/images/200853mask.png already exists.
Skipped: ../public/data/images/771001mask.png already exists.
Skipped: ../public/data/images/770984mask.png already exists.
Skipped: ../public/data/images/771002mask.png already exists.
Skipped: ../public/data/images/47626mask.png already exists.
Skipped: ../public/data/images/42215mask.png already exists.
Skipped: ../public/data/images/42220mask.png already exists.
Skipped: ../public/data/images/50236mask.png already exists.
Skipped: ../pub

Device set to use mps:0
Processing batches: 100%|██████████| 227/227 [30:34<00:00,  8.08s/it]

Error processing object 238928: cannot identify image file <_io.BytesIO object at 0x3572b3ce0>
Skipped: ../public/data/images/250258mask.png already exists.
Skipped: ../public/data/images/243971mask.png already exists.
Skipped: ../public/data/images/52938mask.png already exists.
Skipped: ../public/data/images/52937mask.png already exists.
Skipped: ../public/data/images/550071mask.png already exists.
Skipped: ../public/data/images/549375mask.png already exists.
Skipped: ../public/data/images/248849mask.png already exists.
Skipped: ../public/data/images/204620mask.png already exists.
Skipped: ../public/data/images/204619mask.png already exists.
